## Questions to pick from
1. What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
2. We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.
3. We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability.

#### Question: What time are players most likely to log in during the day and which month is most active?
#### Predictive: Can we predict when players are most likely to log on based on skill level?

Variables to uses: 
- start_time (separate this into month and hour)
#### How to wrangle data
- Make sure all data is from 2024
- From this data we can see which **month is the most popular** (*probably summer*),  and what **hour** has the most players
- Data involving specific players is less important

#### 1. Data description
- number of observations:

sessions.csv = 1535 rows × 5 columns --> only start_time is important (and hashedEmail, but only for merging)

players.csv = 196 rows x 9 columns --> only experience is important (and hashedEmail, but only for merging)

  Provided dataset for player sessions has 1535 rows of observations and 5 variable columns. For sessions dataset, there is a categorical variable named "hashedData" that provides player identification (categorical) for each unique login seen in column "start_time" that provides day, month, year and exact hour (temporal). Out of the remaining columns, "end_time" represents the end of the session (temporal), while "original_start_time" and "original_end_time" most likely represent the exact second (since server has been opened) player has logged in and out. Provided dataset for player information has 196 rows of observations and 7 variable columns. For player dataset, there is the same categorical variable providing player ID names "hashedData" and associated experience level for each player labeled "experience" (also categorical). Dataset provides additonal categorical data; subscription status to newsletter, name, gender, and age. The only qualitative data is "played_hours". There are potential issues with player dataset, as experience level is self reported and could be inaccurate. Additionally, there is no further information provided for player activity beyond login information (e.g., is player contributing a significant amount of activity, requiring larger server capacity).

- issues with the data:
    - Experience is self reported and might not be accurate, no further information regarding activity of player (how active is the player being? is this much activity making the server run poorly? or are they not active at all? inflated login count but little activity, did not need to upgrade server to handle a lot of activity during certain period of time.
#### 2. Question
- what is the response variable of interest and one or more explanatory variables

Response variable: when would player be the most active

Explanatory variables: experience level

The question of interest that will be answered is question 3, regarding demand forecasting.

    Can we predict when players are most likely to log on based on skill level?
- describe how data will help address question of interest

Since the dataset has player login time (start_time) and player skill level (experience), we can measure when players are most active and specifically what type of player is most active. The data can be categorized by experience, allowing us to see what percentage of players in each skill level are playing at a given time. This information can then be applied to anticipate peak player activity if there is a surge in a particular amount of players of a certain experience level.
#### 3. Exploratory data analysis and visualization
- data set loaded into python
- wrangle data (minimum)
- visualizations (include labels, titles, and units of measurement)
- what insights do you gain from plots?
  
From plotted graphs, we can see a linear relationship from afternoon to evening and morning to afternoon for player logins.
#### 4. Methods and plan
To address the predictive question, a linear regression model will be performed. This method is appropriate because data shows a linear relationship between variables, making linear regression the better choice over K-Nearest Neighbors regression. After aggregating login information by hour of day/month, the data shows a linear trend (i.e., login number rises from afternoon to evening, and declines from morning to afternoon). Linear regression provides an easily interpretable equation, describing how login likelihood varies with skill level. For this method, we are assuming that the data is linear (e.g., login frequency as a positive or negative linear relationship with experience level and periods of time). A potential weakness of the model selected are that player login patterns might have unexpected peaks (e.g., login spike in the afternoon) that cannot by captured by linear regression. Additionally, there can be outliers that affect predictions (e.g., player logs in multiple times in the afternoon, disrupting the linearity of data from the afternoon to evening). To comapre and select the model, it will be evaluated using Root Mean Squared Error (RMSE) metric to show the differences between predicted data and actual data. Furthermore, we can compare linear regression with KNN regression to see which model has better predictions. To process the data and apply it to the model, we need to encode "experience" as a categorical variable and extract start_hour and start_month from start_time data. The data would be split 70%-30% into training and testing datasets after necessary variables have been wrangled.

In [59]:
# coding to look at month/day activity

import pandas as pd
import altair as alt

month_order = [
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'
]

sessions_raw = pd.read_csv('project_data/sessions.csv')
sessions_times = sessions_raw.drop(['hashedEmail', 'original_start_time', 'original_end_time'], axis = 1)

# organizing data 
data = {'date_column_start': sessions_times['start_time'], 
        'date_column_end': sessions_times['end_time']}

sessions_times = pd.DataFrame(data)

# convert to datetime
sessions_times['date_column_start'] = pd.to_datetime(sessions_times['date_column_start'], dayfirst = True)
sessions_times['date_column_end'] = pd.to_datetime(sessions_times['date_column_end'], dayfirst = True)

# data has been separated by year month and time in day for start data
sessions_times['start_year'] = sessions_times['date_column_start'].dt.year
sessions_times['start_month'] = sessions_times['date_column_start'].dt.month
sessions_times['start_time'] = sessions_times['date_column_start'].dt.hour

# only need start time data, but left end time hour incase we want to look at length of session
sessions_times['end_time'] = sessions_times['date_column_end'].dt.hour

# chronological order of months
sessions_times = sessions_times.sort_values('start_month')

# dropping columns
sessions_new = sessions_times.drop(['date_column_start', 'date_column_end'], axis = 1)

# all data is already from 2024!
  #year2024 = [2024]
  #sessions_wr = sessions_new[sessions_new['start_year'].isin(year2024)]

# tallying months repeated, to see player logins
sessions_log_month = (
    sessions_new['start_month']
    .value_counts()
    .reset_index()
)
sessions_log_month.columns = ['month', 'num_logins']
sessions_log_month['month'] = pd.to_datetime(sessions_log_month['month'], format = '%m').dt.month_name()

sessions_log_month_plot = alt.Chart(sessions_log_month).mark_bar(color = '#CE71D9').encode(
    x = alt.X('month', axis = alt.Axis(labelAngle = -45), sort = month_order).title('Month'),
    y = alt.Y('num_logins:Q').title('Amount of Logins')
).properties(
    width = 350
).configure_axis(
    labelFontSize = 12,
    titleFontSize = 15,
)

# tallying hours repeated, to see player logins
sessions_log_hour = (
    sessions_new['start_time']
    .value_counts()
    .reset_index()
)
sessions_log_hour.columns = ['hour', 'num_logins']

sessions_log_hour_plot = alt.Chart(sessions_log_hour).mark_bar(color = '#71BDD9').encode(
    x = alt.X('hour').title('Hour of the Day (24h clock)'),
    y = alt.Y('num_logins:Q').title('Amount of Logins'),
).properties(
    width = 350
).configure_axis(
    labelFontSize = 12,
    titleFontSize = 15,
)

In [2]:
sessions_log_hour_plot

alt.Chart(...)

In [3]:
sessions_log_month_plot

alt.Chart(...)

In [61]:
# coding to organize hashedEmail and connect data, what player skill level logs on when?
# x axis: time (month or day), y axis: amount of logins, 
# numerical values to categorize skill level bar graph (what percentage of players are what skill level?)

# removing variables not needed
players_raw = pd.read_csv('project_data/players.csv')
players_exp = players_raw.drop(['subscribe', 'played_hours', 'name', 'gender', 'age', 'individualId', 'organizationName'], axis = 1)

# sessions data with hashedEmail
sessions_hash = sessions_raw.drop(['original_start_time', 'original_end_time', 'end_time'], axis = 1)
sessions_hash

# merging dataframe because they share the variable hashedEmail!
hash_merge = pd.merge(sessions_hash, players_exp, on = 'hashedEmail', how = 'inner')
# on: the column or list of columns to join on (exists in both DataFrames)
# how: specifies the type of merge to perform
    # 'inner' (default): returns only rows with matching keys in both DataFrames

# like previous code, i need to seperate start_time and convert to datetime
hash_merge['start_time'] = pd.to_datetime(hash_merge['start_time'], dayfirst = True)

hash_merge['start_month'] = hash_merge['start_time'].dt.month
hash_merge['start_hour'] = hash_merge['start_time'].dt.hour

# per hour
logins_hour = (
    hash_merge.groupby(['start_hour', 'experience'])
    .size()
    .reset_index(name = 'num_logins_hour')
)

logins_hour_plot = alt.Chart(logins_hour).mark_bar().encode(
    x = alt.X('start_hour').title('Hour of the Day (24h clock)'),
    y = alt.Y('num_logins_hour:Q').title('Amount of Logins'),
    color = 'experience:N'
).properties(
    width = 350
).configure_axis(
    labelFontSize = 12,
    titleFontSize = 15,
)

# per month
logins_month = (
    hash_merge.groupby(['start_month', 'experience'])
    .size()
    .reset_index(name = 'num_logins_month')
)

#turn month number to name to make graph understandable
logins_month.columns = ['start_month', 'experience', 'num_logins_month']
logins_month['start_month'] = pd.to_datetime(logins_month['start_month'], format = '%m').dt.month_name()

month_order = [
    'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'
]

logins_month_plot = alt.Chart(logins_month).mark_bar().encode(
    x = alt.X('start_month', axis = alt.Axis(labelAngle = -45), sort = month_order).title('Month'),
    y = alt.Y('num_logins_month:Q').title('Amount of logins'),
    color = 'experience:N'
).properties(
    width = 350
).configure_axis(
    labelFontSize = 12,
    titleFontSize = 15,
)

In [56]:
logins_month_plot

alt.Chart(...)

In [40]:
logins_hour_plot

alt.Chart(...)